# SQL Alchemy

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy.exc import SQLAlchemyError

# Step 1: Define the database connection
# Replace 'sqlite:///example.db' with your database URI
DATABASE_URI = "sqlite:///example.db"  # SQLite database for simplicity
engine = create_engine(DATABASE_URI, echo=True)

# Step 2: Define the Base class
Base = declarative_base()

# Step 3: Define the models (tables)
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, nullable=False)
    age = Column(Integer, nullable=False)
    salary = Column(Float, default=0.0)

    # Relationship with Address table
    addresses = relationship("Address", backref=backref("user", lazy=True))

class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True, autoincrement=True)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    address = Column(String, nullable=False)

# Step 4: Create tables
Base.metadata.create_all(engine)

# Step 5: Set up the session
Session = sessionmaker(bind=engine)
session = Session()

try:
    # Step 6: Insert data into tables
    user1 = User(name="Alice", age=30, salary=60000.0)
    user2 = User(name="Bob", age=25, salary=50000.0)
    session.add_all([user1, user2])
    session.commit()

    # Add addresses for users
    address1 = Address(user_id=user1.id, address="123 Main St")
    address2 = Address(user_id=user2.id, address="456 Elm St")
    session.add_all([address1, address2])
    session.commit()

    # Step 7: Query data
    print("\nAll Users:")
    for user in session.query(User).all():
        print(f"ID: {user.id}, Name: {user.name}, Age: {user.age}, Salary: {user.salary}")

    print("\nUsers with age > 25:")
    for user in session.query(User).filter(User.age > 25).all():
        print(f"Name: {user.name}, Age: {user.age}")

    # Query with join
    print("\nUser Addresses:")
    results = session.query(User, Address).join(Address).all()
    for user, address in results:
        print(f"User: {user.name}, Address: {address.address}")

    # Using SQL functions
    avg_salary = session.query(func.avg(User.salary)).scalar()
    print(f"\nAverage Salary: {avg_salary}")

    # Step 8: Update data
    user_to_update = session.query(User).filter_by(name="Alice").first()
    if user_to_update:
        user_to_update.salary = 65000.0
        session.commit()

    print("\nUpdated Users:")
    for user in session.query(User).all():
        print(f"Name: {user.name}, Salary: {user.salary}")

    # Step 9: Delete data
    user_to_delete = session.query(User).filter_by(name="Bob").first()
    if user_to_delete:
        session.delete(user_to_delete)
        session.commit()

    print("\nRemaining Users After Deletion:")
    for user in session.query(User).all():
        print(f"Name: {user.name}")

except SQLAlchemyError as e:
    # Handle errors and rollback
    session.rollback()
    print(f"An error occurred: {e}")
finally:
    # Step 10: Clean up
    session.close()

# Step 11: Drop all tables (optional, for cleanup)
# Uncomment to drop all tables
# Base.metadata.drop_all(engine)
